# MongoDB Basic

- Create a new MongoClient instance
- db = test-database 
- number of records in a collection
- insert item
- delete item

In [16]:
import pymongo
client = pymongo.MongoClient('localhost',27017)
db=client['test-database']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test-database')

- collection= 'test-collection'under db
- return the number of recored in collection

In [20]:
collection = db['test-collection']
collection.count()

4

- create a dict named 'test_data'
- insert 'test_data' into collection

In [21]:
test_data={'Age':39, 'Gender':'M','Occupation':'Graphical Designer'}
print("test_data",test_data)
insert_result= collection.insert_one(test_data)
print("insert_result: ",insert_result)

test_data {'Age': 39, 'Gender': 'M', 'Occupation': 'Graphical Designer'}
insert_result:  <pymongo.results.InsertOneResult object at 0x000001A3FB01DF08>


if I have a line that writes to the DB, "acknowledged" write concern mode would be True

In [22]:
print("acknowledged: ",insert_result.acknowledged)
print("record_count: ",collection.count())

acknowledged:  True
record_count:  5


check the insert_ID of inserted item

In [24]:
insert_result.inserted_id

ObjectId('5ee8dd1c52916cce5a5ede00')

delete one result 

In [25]:
delete_result = collection.delete_one({'Age':39})

In [26]:
collection.count()

4

# Insert and Quality Check

- create client
- create db
- create collection
- mapping the columns heading and value as a dict
- insert each dict as a row in the collection

**check the type of value (string to int) **

In [30]:
import pymongo
import pandas as pd
client = pymongo.MongoClient('localhost',27017)
# create new database - test
db = client['test']
# #create new collection - income
collection = db['income']

Batch insert

In [34]:
with open('data\income_header.txt') as f_in:
    columns_heading = f_in.readline()
columns_heading_list = columns_heading.split(',')
columns_heading_list

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'label']

In [36]:

with open('data\income.txt') as f_in:
    for line in f_in:
        row_list = line.rstrip('\n').split(',')
        row_dict = dict(zip(columns_heading_list,row_list))
        try:
            row_dict['age']=int(row_dict['age'])
            collection.insert_one(row_dict)
        except:
            pass

In [37]:
collection.count()

32561

In [ ]:
#cursor is a data structure that does several things including: keeping a pointer to the current row in a results set.

In [38]:
age_over_35 = collection.find({'age':{'$gt':35}})
print(type(age_over_35))

<class 'pymongo.cursor.Cursor'>


In [39]:
print("The next result in the list: \n",age_over_35.next())
print("\nCount of client with age over 35:\n",age_over_35.count())

The next result in the list: 
 {'_id': ObjectId('5ee8e21c52916cce5a5ede05'), 'age': 39, 'workclass': ' State-gov', 'fnlwgt': ' 77516', 'education': ' Bachelors', 'education-num': ' 13', 'marital-status': ' Never-married', 'occupation': ' Adm-clerical', 'relationship': ' Not-in-family', 'race': ' White', 'sex': ' Male', 'capital-gain': ' 2174', 'capital-loss': ' 0', 'hours-per-week': ' 40', 'native-country': ' United-States', 'label': ' <=50K'}

Count of client with age over 35:
 17636


# Index data

In [42]:
age_morethan_25 = collection.find({'age':{'$gt':25}})
print(age_morethan_25.next())

{'_id': ObjectId('5ee8e21c52916cce5a5ede05'), 'age': 39, 'workclass': ' State-gov', 'fnlwgt': ' 77516', 'education': ' Bachelors', 'education-num': ' 13', 'marital-status': ' Never-married', 'occupation': ' Adm-clerical', 'relationship': ' Not-in-family', 'race': ' White', 'sex': ' Male', 'capital-gain': ' 2174', 'capital-loss': ' 0', 'hours-per-week': ' 40', 'native-country': ' United-States', 'label': ' <=50K'}


In [43]:
index_result=db.profiles.create_index([('age',pymongo.ASCENDING)],unique=False)

In [44]:
index_result

'age_1'

# PyMongo and Pandas

In [46]:
import pymongo
import pandas as pd
client = pymongo.MongoClient('localhost',27017)
db = client['test']
collection = db['income']

In [50]:
income_df=pd.DataFrame(list(collection.find()))
income_df

,_id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,5ee8e21c52916cce5a5ede05,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,5ee8e21c52916cce5a5ede06,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,5ee8e21c52916cce5a5ede07,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,5ee8e21c52916cce5a5ede08,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,5ee8e21c52916cce5a5ede09,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,5ee8e22b52916cce5a5f5d31,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,5ee8e22b52916cce5a5f5d32,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,5ee8e22b52916cce5a5f5d33,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,5ee8e22b52916cce5a5f5d34,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [51]:
income_df.age.describe()

count    32561.000000
mean        38.581647
std         13.640433
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64

In [52]:
income_df['native-country'].value_counts()

 United-States                 29170
 Mexico                          643
 ?                               583
 Philippines                     198
 Germany                         137
 Canada                          121
 Puerto-Rico                     114
 El-Salvador                     106
 India                           100
 Cuba                             95
 England                          90
 Jamaica                          81
 South                            80
 China                            75
 Italy                            73
 Dominican-Republic               70
 Vietnam                          67
 Guatemala                        64
 Japan                            62
 Poland                           60
 Columbia                         59
 Taiwan                           51
 Haiti                            44
 Iran                             43
 Portugal                         37
 Nicaragua                        34
 Peru                             31
 